## Making init files:

- need air temperature

- need rivers deltat

- need both boundary conditions. 

- need to interpolate deltat's onto our depth grid

In [2]:
## 
import arrow
import netCDF4 as nc
import glob
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy

## atmospheric files

downloaded from:
/home/tjarnik/projects/def-allen/SalishSea/forcing/atmospheric/GEM2.5/operational/FOR_TEPLO

In [3]:
tair_delta = 2.12

In [7]:
w = nc.Dataset('./FOR_TEPLO/ops_y2017m09d04.nc')
print(w)
print(w['solar'])
print(w['therm_rad'])

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    History: Mon Sep  4 11:56:49 2017: ncks -4 -L4 -O /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m09d04.nc /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m09d04.nc
created by wgrib2
    GRIB2_grid_template: 20
    NCO: 4.4.2
    dimensions(sizes): time_counter(24), y(266), x(256)
    variables(dimensions): float32 atmpres(time_counter,y,x), float64 nav_lat(y,x), float64 nav_lon(y,x), float32 percentcloud(time_counter,y,x), float32 precip(time_counter,y,x), float32 qair(time_counter,y,x), float32 solar(time_counter,y,x), float32 tair(time_counter,y,x), float32 therm_rad(time_counter,y,x), float64 time_counter(time_counter), float32 u_wind(time_counter,y,x), float32 v_wind(time_counter,y,x), float64 x(x), float64 y(y)
    groups: 

<class 'netCDF4._netCDF4.Variable'>
float32 solar(time_counter, y, x)
    _FillValue: 9.999e+20
    short_name: DSWRF_surface


In [5]:
def dilley_unsworth(lat, rpn_vp_on_ops, rpn_NT_on_ops, rpn_TT_on_ops, sigma):
    
    # rpn_TT_on_ops >> temperature in kelvin
    # 
# https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2008WR007394
# Dilley clear sky algorithm
    eo = rpn_vp_on_ops/1000.
    To = rpn_TT_on_ops
    w = 465*eo/To
    Lclr = 59.38 + 113.7*(To/273.16)**6 + 96.96*np.sqrt(w/2.5) # Dilley
    eclr = Lclr/(sigma * To**4)

    # Unsworth
    cf = rpn_NT_on_ops
    ewc = (1 - 0.84*cf)*eclr + 0.84*cf
#
    Lwc = ewc * sigma * To**4
    return Lwc